# Generate images
> Use the OpenAI API to make banchan images from the dish names.

---

#### Import Python tools and Jupyter config

In [1]:
import os
import time
import requests
import unicodedata
import pandas as pd
import jupyter_black
from PIL import Image
from io import BytesIO
from openai import OpenAI
from tqdm import tqdm

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [3]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key=os.environ.get("OPEN_AI"))

---

## Generate

In [4]:
# Read dishes from scraper notebook
df = pd.read_json("data/processed/banchan_dishes.json")

In [5]:
def generate_image(dish_name, korean_name, phonetic_name):
    try:
        response = client.images.generate(
            model="dall-e-3",
            prompt=f"A photorealistic image of {dish_name}, a Korean banchan dish called {korean_name} or {phonetic_name}, served in small portions on a white plate in a minimalistic setting. The image should be highly detailed and realistic, as if taken with a professional camera.",
            size="1024x1024",
            quality="standard",
            n=1,
        )
        return response.data[0].url
    except Exception as e:
        print(f"Error generating image for {dish_name}: {e}")
        return None

In [6]:
# Function to remove accents from dish names, e.g., sautéed
def remove_accents(input_str):
    return "".join(
        c
        for c in unicodedata.normalize("NFKD", input_str)
        if unicodedata.category(c) != "Mn"
    )


# Function to resize and save a thumbnail (150px width)
def save_thumbnail(img, clean_dish_name):
    try:
        # Resize the image to 150px width, maintaining aspect ratio
        thumbnail = img.copy()
        thumbnail.thumbnail((150, 150))  # 150px width, height adjusts automatically

        # Ensure thumbnails directory exists
        os.makedirs("images/thumbnails", exist_ok=True)

        # Save the thumbnail in the thumbnails directory
        thumbnail.save(f"images/thumbnails/{clean_dish_name}.png")
        # print(f"Saved thumbnail for {clean_dish_name}")
    except Exception as e:
        print(f"Error saving thumbnail for {clean_dish_name}: {e}")


# Function to download and save images locally, including a thumbnail version
def save_image(url, dish_name):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))

        # Clean dish name: lowercase, replace spaces with underscores and remove accents
        clean_dish_name = remove_accents(dish_name.lower().replace(" ", "_"))

        # Save the full-size image
        img.save(f"images/{clean_dish_name}.png")
        # print(f"Saved image for {dish_name}")

        # Save the thumbnail (150px width)
        save_thumbnail(img, clean_dish_name)

    except Exception as e:
        print(f"Error saving image for {dish_name}: {e}")

In [7]:
# Loop through the DataFrame and generate images, adding the image URL back to the DataFrame
for index, row in tqdm(df.iterrows(), total=len(df)):
    image_url = generate_image(row["dish"], row["korean"], row["phonetic"])
    if image_url:
        # Add the image URL directly to the corresponding row in the DataFrame
        df.at[index, "image_url"] = image_url

        # Download and save the full-size image and thumbnail
        save_image(image_url, row["dish"])

 19%|██████████████████████████▉                                                                                                                      | 39/210 [12:34<44:33, 15.64s/it]

Error generating image for Vegetarian kimchi: Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'Your request was rejected as a result of our safety system. Your prompt may contain text that is not allowed by our safety system.', 'param': None, 'type': 'invalid_request_error'}}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 210/210 [1:09:48<00:00, 19.95s/it]


---

## Export

#### CSV

In [8]:
df.to_csv("data/processed/banchan_dishes_images.csv", index=False)

#### JSON

In [9]:
df.to_json("data/processed/banchan_dishes_images.json", orient="records", index=4)